In [2]:
import numpy as np
import pandas as pd

RAW_PATH = '/content/drive/MyDrive/data/instacart-market-basket-analysis/raw/'
PREPARED_PATH = '/content/drive/MyDrive/data/instacart-market-basket-analysis/prepared/'

In [3]:
def reduce_memory(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    for col in df.columns:
        if df[col].dtypes in ["int64", "int32", "int16"]:
            cmin = df[col].min()
            cmax = df[col].max()
            if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        if df[col].dtypes in ["float64", "float32"]:
            cmin = df[col].min()
            cmax = df[col].max()
            if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
    print("")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df

# 1. 피쳐 생성

In [ ]:
orders = pd.read_csv(RAW_PATH + 'orders.csv', dtype={
                            'order_id': np.uint32,              # 1 ~ 3421083
                            'user_id': np.uint32,               # 1 ~ 206209
                            'eval_set': 'category',             # 3가지
                            'order_number': np.uint8,           # 1 ~ 100
                            'order_dow': np.uint8,              # 0 ~ 6 
                            'order_hour_of_day': np.uint8,      # 0 ~ 23
                            'days_since_prior_order': np.float32})
priors = pd.read_csv(RAW_PATH + 'order_products__prior.csv', dtype={
                            'order_id': np.uint32,          # 1 ~ 3421083
                            'product_id': np.uint16,        # 1 ~ 49688
                            'add_to_cart_order': np.uint8,  # 1 ~ 80 
                            'reordered': np.uint8})         # 0 ~ 1
train = pd.read_csv(RAW_PATH + 'order_products__train.csv', dtype={
                            'order_id': np.uint32,          # 1 ~ 3421083
                            'product_id': np.uint16,        # 1 ~ 49688
                            'add_to_cart_order': np.uint8,  # 1 ~ 80 
                            'reordered': np.uint8})         # 0 ~ 1
products = pd.read_csv(RAW_PATH + 'products.csv', dtype={
                              'product_id': np.uint16,     # 1 ~ 49688
                              'aisle_id': np.uint8,        # 1 ~ 134
                              'department_id': np.uint8}) # 1 ~ 21
aisles = pd.read_csv(RAW_PATH + 'aisles.csv', dtype={'product_id': np.uint16})
departments = pd.read_csv(RAW_PATH + 'departments.csv', dtype={'product_id': np.uint16})

In [ ]:
# 이전 구매내역에 주문 정보 추가
prior_df = priors.merge(orders, on='order_id', how='left')
prior_df = prior_df.merge(products, on='product_id', how='left')

print(prior_df.shape)
prior_df.head()

In [ ]:
prior_df.to_pickle(PREPARED_PATH + 'prior_df.pkl')

## 1.1 주문 정보

이전(prior) 주문별 정보

---
    items : 주문당 구매 제품 수
    reorder_in_order : 주문당 재주문 제품 비율

In [ ]:
prior_df = pd.read_pickle(PREPARED_PATH + 'prior_df.pkl')
orders = pd.read_csv(RAW_PATH + 'orders.csv', dtype={
                            'order_id': np.uint32,              # 1 ~ 3421083
                            'user_id': np.uint32,               # 1 ~ 206209
                            'eval_set': 'category',             # 3가지
                            'order_number': np.uint8,           # 1 ~ 100
                            'order_dow': np.uint8,              # 0 ~ 6 
                            'order_hour_of_day': np.uint8,      # 0 ~ 23
                            'days_since_prior_order': np.float32})

prior_order_feats = prior_df.groupby(['user_id','order_number']).agg(basket_size=('product_id','count'), 
                                                                     reorder_in_order=('reordered', 'mean'))
prior_order_feats.reset_index(inplace=True)
prior_order_feats = prior_order_feats.merge(orders.drop('eval_set', axis=1), on=['user_id','order_number'], how='left')
col = ['user_id', 'order_number', 'basket_size','reorder_in_order', 'order_dow','order_hour_of_day', 'days_since_prior_order', 'order_id']
prior_order_feats = prior_order_feats[col]

del orders

print(prior_order_feats.shape)
prior_order_feats.head()

(3214874, 8)


,user_id,order_number,basket_size,reorder_in_order,order_dow,order_hour_of_day,days_since_prior_order,order_id
0,1,1,5,0.000,2,8,NaN,2539329
1,1,2,6,0.500,3,7,15.0,2398795
2,1,3,5,0.600,3,12,21.0,473747
3,1,4,5,1.000,4,7,29.0,2254736
4,1,5,8,0.625,4,15,28.0,431534


In [ ]:
prior_order_feats.to_pickle(PREPARED_PATH + 'prior_order_feats.pkl')

In [ ]:
# 마지막 주문 3개 뽑기
last_three_orders = prior_order_feats.sort_values('order_number', ascending=False)\
                    .groupby('user_id').head(3).sort_values(['user_id', 'order_number'], ignore_index=True)
last_three_orders['rank'] = last_three_orders.groupby('user_id')['order_number'].rank()

print(last_three_orders.shape)
last_three_orders.head()

(618627, 9)


,user_id,order_number,basket_size,reorder_in_order,order_dow,order_hour_of_day,days_since_prior_order,order_id,rank
0,1,8,6,0.666667,1,14,14.0,3108588,1.0
1,1,9,6,1.000000,1,16,0.0,2295261,2.0
2,1,10,9,0.666667,4,8,30.0,2550362,3.0
3,2,12,19,0.578947,1,9,28.0,3186735,1.0
4,2,13,9,0.000000,4,11,30.0,3268552,2.0


In [ ]:
last_three_orders.to_pickle(PREPARED_PATH + 'last_three_orders.pkl')

## 1.2 유저 정보

유저별 정보

---
    U_total_orders : 총 주문 수
    U_total_products : 총 구매 제품 수
    U_unique_products : 총 구매 제품 종류 수
    U_total_reorders : 총 재주문 수
    U_unique_reordered_products : 총 재주문 제품 종류 수
    U_avg_basket_size : 주문당 평균 구매 제품 수
    U_avg_reorder_in_order : 주문당 평균 재구매 제품 비율
    U_order_days : 총 주문 기간
    U_basket_size_order : 마지막 3개의 주문 구매 제품 수
    U_re_in_order : 마지막 3개의 주문 재주문 제품 포함 비율

In [ ]:
prior_df = pd.read_pickle(PREPARED_PATH + 'prior_df.pkl')

user_feats = prior_df.groupby('user_id').agg(U_total_orders=('order_number', 'max'), 
                                             U_total_products=('product_id', 'count'), 
                                             U_unique_products=('product_id', 'nunique'),
                                             U_total_reorders=('reordered', 'sum'))
user_feats['U_unique_reordered_products'] = prior_df[prior_df.reordered == 1].drop_duplicates(['user_id', 'product_id']).groupby('user_id').product_id.nunique()
user_feats['U_unique_reordered_products'].fillna(0, inplace=True)
user_feats['U_avg_basket_size'] = prior_order_feats.groupby('user_id')['basket_size'].mean()
user_feats['U_avg_reorder_in_order'] = prior_order_feats[prior_order_feats['order_number'] != 1].groupby('user_id')['reorder_in_order'].mean()

user_feats['U_order_days'] = prior_order_feats.groupby('user_id').days_since_prior_order.sum()

user_feats.reset_index(inplace=True)
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], 
                                                 values=['basket_size', 'reorder_in_order']).\
                                                 reset_index(drop = False)
last_order_feats.columns = ['user_id','U_basket_size_order_3', 'U_basket_size_order_2', 'U_basket_size_order_1', 'U_re_in_order_3', 'U_re_in_order_2', 'U_re_in_order_1']
user_feats = user_feats.merge(last_order_feats, on='user_id', how='left')

print(user_feats.shape)
user_feats.head()

(206209, 15)


,user_id,U_total_orders,U_total_products,U_unique_products,U_total_reorders,U_unique_reordered_products,U_avg_basket_size,U_avg_reorder_in_order,U_order_days,U_basket_size_order_3,U_basket_size_order_2,U_basket_size_order_1,U_re_in_order_3,U_re_in_order_2,U_re_in_order_1
0,1,10,59,18,41.0,10.0,5.900000,0.784259,176.0,6,6,9,0.666667,1.0,0.666667
1,2,14,195,102,93.0,37.0,13.928571,0.482419,198.0,19,9,16,0.578947,0.0,0.625000
2,3,12,88,33,55.0,19.0,7.333333,0.718710,133.0,6,5,6,0.833333,1.0,1.000000
3,4,5,18,17,1.0,1.0,3.600000,0.035714,55.0,7,2,3,0.142857,0.0,0.000000
4,5,4,37,23,14.0,8.0,9.250000,0.503704,40.0,9,5,12,0.444444,0.4,0.666667


In [ ]:
user_feats.to_pickle(PREPARED_PATH + 'user_feats.pkl')

## 1.3 유저별 구매내역



유저별 구매내역 정보

---
    UP_total_orders : 총 주문 수
    UP_total_reorders : 총 재주문 수
    UP_mean_cart_order : 평균 장바구니 순위
    U_total_orders : 유저 총 주문 수
    UP_reorder_ratio : 해당 제품을 처음 구매한 이후 재주문이 가능한 기간(주문 수)동안의 재주문 율
    UP_order_percentage : 유저의 전체 주문 중 해당 제품을 구매한 비율
    UP_order_days : 해당 제품을 구매한 기간
    UP_mean_days_since_prior_order : 평균 재주문 기간
    order-3,2,1 : 마지막 3번의 주문에서 재주문 여부 (구매 된적이 없다면 -1)

In [ ]:
user_feats = pd.read_pickle(PREPARED_PATH + 'user_feats.pkl')
prior_df = pd.read_pickle(PREPARED_PATH + 'prior_df.pkl')
last_three_orders = pd.read_pickle(PREPARED_PATH + 'last_three_orders.pkl')
products = pd.read_csv(RAW_PATH + 'products.csv', dtype={'product_id': np.uint16, 'aisle_id': np.uint8, 'department_id': np.uint8}) 

user_prod_feats = prior_df.groupby(['user_id', 'product_id']).agg(UP_total_orders=('reordered', 'count'),
                                                                  UP_total_reorders=('reordered', 'sum'),
                                                                  UP_first_order_num=('order_number', 'min'),
                                                                  UP_mean_cart_order=('add_to_cart_order', 'mean'))
user_prod_feats.reset_index(inplace=True)

# 재구매 가능 기간 재주문율 추가
user_prod_feats = user_prod_feats.merge(user_feats[['user_id', 'U_total_orders']], on='user_id', how='left')
user_prod_feats['order_range_D'] = user_prod_feats.U_total_orders - user_prod_feats.UP_first_order_num
user_prod_feats['UP_reorder_ratio'] = user_prod_feats.UP_total_reorders / user_prod_feats['order_range_D']
user_prod_feats.UP_reorder_ratio.fillna(0, inplace=True)

# 주문 율
user_prod_feats['UP_order_percentage'] = user_prod_feats.UP_total_orders / user_prod_feats.U_total_orders

# 재주문 기간
up_dspo = prior_df[prior_df.reordered == 1].groupby(['user_id', 'product_id']).days_since_prior_order.sum().reset_index().rename(columns={'days_since_prior_order': 'UP_order_days'})
user_prod_feats = user_prod_feats.merge(up_dspo, on=['user_id', 'product_id'], how='left')
user_prod_feats['UP_mean_days_since_prior_order'] = user_prod_feats.UP_order_days / user_prod_feats.UP_total_reorders
del up_dspo

# 마지막 3번의 주문에서 재구매여부
last_three_orders_prods = prior_df.merge(last_three_orders[['user_id', 'order_number', 'rank']], on=['user_id', 'order_number'], how='inner')
product_purchase_history = last_three_orders_prods.pivot_table(index = ['user_id', 'product_id'],\
                                                               columns='rank', values = 'reordered')
product_purchase_history.columns = ['UP_order_3', 'UP_order_2', 'UP_order_1']
product_purchase_history.reset_index(inplace=True)
user_prod_feats = user_prod_feats.merge(product_purchase_history, on=['user_id', 'product_id'], how='left')
del last_three_orders, last_three_orders_prods, product_purchase_history

user_prod_feats = user_prod_feats.merge(products, on='product_id', how='left')

user_prod_feats.fillna(-1, inplace=True)

user_prod_feats.sort_values(['user_id', 'UP_total_reorders'], ascending=[True, False], ignore_index=True, inplace=True)
user_prod_feats = user_prod_feats.astype({'UP_total_orders':np.int8, 
                                          'UP_mean_cart_order':np.float32,
                                          'UP_reorder_ratio': np.float32, 
                                          'UP_order_percentage' : np.float32, 
                                          'UP_order_days' : np.int16,   
                                          'UP_order_3': np.int8, 
                                          'UP_order_2': np.int8, 
                                          'UP_order_1': np.int8})

user_prod_feats.drop(['order_range_D', 'UP_first_order_num'], axis=1, inplace=True)

print(user_prod_feats.shape)
user_prod_feats.head()

(13307953, 16)


,user_id,product_id,UP_total_orders,UP_total_reorders,UP_mean_cart_order,U_total_orders,UP_reorder_ratio,UP_order_percentage,UP_order_days,UP_mean_days_since_prior_order,UP_order_3,UP_order_2,UP_order_1,product_name,aisle_id,department_id
0,1,196,10,9,1.400000,10,1.00,1.0,176,19.555555,1,1,1,Soda,77,7
1,1,12427,10,9,3.300000,10,1.00,1.0,176,19.555555,1,1,1,Original Beef Jerky,23,19
2,1,10258,9,8,3.333333,10,1.00,0.9,161,20.125000,1,1,1,Pistachios,117,19
3,1,25133,8,7,4.000000,10,1.00,0.8,140,20.000000,1,1,1,Organic String Cheese,21,16
4,1,13032,3,2,6.333333,10,0.25,0.3,50,25.000000,-1,-1,1,Cinnamon Toast Crunch,121,14


In [ ]:
user_prod_feats.to_pickle(PREPARED_PATH + 'user_prod_feats.pkl')

## 1.4 제품 정보

전체 주문 제품 통계

---
    P_total_orders : 총 판매 수 
    P_unique_users : 구매한 유저 수
    P_total_reorders : 총 재주문 수 
    P_mean_cart_order : 평균 장바구니 순위
    P_reorder_rate : 재주문율 

    P_sum_order_days : 총 구매 기간
    P_mean_order_days : 평균 구매 기간
    P_days_since_prior_order : 평균 재주문 기간

    is_organic : 유기농 제품 여부
    is_lowfat : 저지방/무지방 제품 여부
    P_keyword : 제품 이름에 들어가는 키워드별로 그룹화 (테스트)

    소분류/대분류 별
    total_orders : 총 판매 수
    total_reorders : 총 재주문 수
    reorder_rate : 재주문 율
    mean_cart_order : 평균 장바구니 순위
    mean_days_since_prior_order : 평균 재주문 기간

### 1.4.1 제품별

In [7]:
word_list_lower =  ["organic", 'fat', "whole", "reduced", 'baby', "water", "banana", "greek", "hass", "strawberry", "honey", 'half']
word_list_upper = ["Organic", 'Fat', "Whole", "Reduced", "Baby", "Water", "Banana", "Greek", "Hass", "Strawberry", "Honey", 'Half']

# 단어가 제품 이름에 포함되면 해당 단어를 반환하는 함수
def keyword(x):
    for i, (word_low, word_up) in enumerate(zip(word_list_lower, word_list_upper)):
        if word_low in x or word_up in x:
            return i
    else :
        return -1

In [8]:
products = pd.read_csv(RAW_PATH + 'products.csv', dtype={'product_id': np.uint16, 'aisle_id': np.uint8, 'department_id': np.uint8}) 
prior_df = pd.read_pickle(PREPARED_PATH + 'prior_df.pkl')

prod_feats = prior_df.groupby('product_id').agg(P_total_orders=('product_id', 'count'), 
                                                P_unique_users=('user_id','nunique'), 
                                                P_total_reorders=('reordered', 'sum'),
                                                P_reorder_rate=('reordered', 'mean'),
                                                P_mean_cart_order=('add_to_cart_order', 'mean'))

prod_feats_from_UP = user_prod_feats.groupby('product_id').agg(P_sum_order_days=('UP_order_days', 'sum'), 
                                                               P_mean_order_days=('UP_order_days', 'mean'), 
                                                               P_days_since_prior_order=('UP_mean_days_since_prior_order', 'mean'))

prod_feats = prod_feats.merge(prod_feats_from_UP, on='product_id', how='left')

prod_feats = prod_feats.merge(products, on='product_id', how='left')

prod_feats['is_organic'] = prod_feats['product_name'].apply(lambda x: 1 if 'Organic' in x else 0)
prod_feats['is_lowfat'] = prod_feats['product_name'].apply(lambda x: 1 if 'fat' in x or 'Fat' in x else 0)
prod_feats['P_keyword'] = prod_feats['product_name'].apply(keyword)

del products, prod_feats_from_UP

prod_feats.sort_values(by='P_total_orders', inplace=True, ascending=False, ignore_index=True)

print(prod_feats.shape)
prod_feats.head()

(49677, 15)


,product_id,P_total_orders,P_unique_users,P_total_reorders,P_reorder_rate,P_mean_cart_order,P_sum_order_days,P_mean_order_days,P_days_since_prior_order,product_name,aisle_id,department_id,is_organic,is_lowfat,P_keyword
0,24852,472565,73956,398609.0,0.843501,4.894129,4174732.0,56.448861,9.997801,Banana,24,4,0,0,6
1,13176,379450,63537,315913.0,0.832555,5.095947,2985543.0,46.989046,8.648257,Bag of Organic Bananas,24,4,1,0,0
2,21137,264683,58838,205845.0,0.777704,7.248902,1902667.0,32.337384,7.392364,Organic Strawberries,24,4,1,0,0
3,21903,241921,55037,186884.0,0.772500,7.429640,1973413.0,35.856115,8.417183,Organic Baby Spinach,123,4,1,0,0
4,47209,213584,43453,170131.0,0.796553,6.775011,1615360.0,37.174879,7.814569,Organic Hass Avocado,24,4,1,0,0


### 1.4.2 소분류 별 (aisle)

In [9]:
# 소분류
aisles = pd.read_csv(RAW_PATH + 'aisles.csv', dtype={'product_id': np.uint16})

aisle_feats = prior_df.groupby('aisle_id', as_index=False).agg(A_total_orders=('reordered', 'count'), 
                                                                A_total_reorders=('reordered', 'sum'), 
                                                                A_reorder_rate=('reordered', 'mean'),
                                                                A_avg_cart_order=('add_to_cart_order', 'mean'))

aisle_feats_from_up = user_prod_feats.groupby('aisle_id', as_index=False).agg(A_mean_days_since_prior_order=('UP_mean_days_since_prior_order', 'mean'))

aisle_feats = aisle_feats.merge(aisle_feats_from_up).merge(aisles)
aisle_feats.sort_values(by='A_total_orders', inplace=True, ascending=False, ignore_index=True)

del aisles
print(aisle_feats.shape)
aisle_feats.head()

(134, 7)


,aisle_id,A_total_orders,A_total_reorders,A_reorder_rate,A_avg_cart_order,A_mean_days_since_prior_order,aisle
0,24,3642188,2615469.0,0.718104,7.144228,6.013567,fresh fruits
1,83,3418021,2032172.0,0.594546,8.852012,4.680626,fresh vegetables
2,123,1765313,1127177.0,0.638514,8.399527,5.108206,packaged vegetables fruits
3,120,1452343,997018.0,0.686489,7.861644,5.439575,yogurt
4,21,979763,573383.0,0.585226,9.078364,4.442752,packaged cheese


### 1.4.3 대분류 별 (department)

In [10]:
# 대분류
departments = pd.read_csv(RAW_PATH + 'departments.csv', dtype={'product_id': np.uint16})
dp_feats = prior_df.groupby('department_id', as_index=False).agg(D_total_orders=('reordered', 'count'), 
                                                                 D_total_reorders=('reordered', 'sum'), 
                                                                 D_reorder_rate=('reordered', 'mean'), 
                                                                 D_avg_cart_order=('add_to_cart_order', 'mean'))

dp_feats_from_up = user_prod_feats.groupby('department_id', as_index=False).agg(D_mean_days_since_prior_order=('UP_mean_days_since_prior_order', 'mean'))

dp_feats = dp_feats.merge(dp_feats_from_up).merge(departments)
dp_feats.sort_values(by='D_total_orders', inplace=True, ascending=False, ignore_index=True)

del departments
print(dp_feats.shape)
dp_feats.head()

(21, 7)


,department_id,D_total_orders,D_total_reorders,D_reorder_rate,D_avg_cart_order,D_mean_days_since_prior_order,department
0,4,9479291,6160710.0,0.649913,8.022875,5.161306,produce
1,16,5414016,3627221.0,0.669969,7.495423,5.249701,dairy eggs
2,19,2887550,1657973.0,0.574180,9.187743,3.943013,snacks
3,7,2690129,1757892.0,0.653460,6.976699,4.778593,beverages
4,1,2236432,1211890.0,0.541885,8.996414,3.989675,frozen


병합




In [11]:
prod_feats = prod_feats.merge(aisle_feats, on='aisle_id', how='left')
prod_feats = prod_feats.merge(dp_feats, on='department_id', how='left')

print(prod_feats.shape)
prod_feats.head()

(49677, 27)


,product_id,P_total_orders,P_unique_users,P_total_reorders,P_reorder_rate,P_mean_cart_order,P_sum_order_days,P_mean_order_days,P_days_since_prior_order,product_name,...,A_reorder_rate,A_avg_cart_order,A_mean_days_since_prior_order,aisle,D_total_orders,D_total_reorders,D_reorder_rate,D_avg_cart_order,D_mean_days_since_prior_order,department
0,24852,472565,73956,398609.0,0.843501,4.894129,4174732.0,56.448861,9.997801,Banana,...,0.718104,7.144228,6.013567,fresh fruits,9479291,6160710.0,0.649913,8.022875,5.161306,produce
1,13176,379450,63537,315913.0,0.832555,5.095947,2985543.0,46.989046,8.648257,Bag of Organic Bananas,...,0.718104,7.144228,6.013567,fresh fruits,9479291,6160710.0,0.649913,8.022875,5.161306,produce
2,21137,264683,58838,205845.0,0.777704,7.248902,1902667.0,32.337384,7.392364,Organic Strawberries,...,0.718104,7.144228,6.013567,fresh fruits,9479291,6160710.0,0.649913,8.022875,5.161306,produce
3,21903,241921,55037,186884.0,0.772500,7.429640,1973413.0,35.856115,8.417183,Organic Baby Spinach,...,0.638514,8.399527,5.108206,packaged vegetables fruits,9479291,6160710.0,0.649913,8.022875,5.161306,produce
4,47209,213584,43453,170131.0,0.796553,6.775011,1615360.0,37.174879,7.814569,Organic Hass Avocado,...,0.718104,7.144228,6.013567,fresh fruits,9479291,6160710.0,0.649913,8.022875,5.161306,produce


In [12]:
prod_feats.to_pickle(PREPARED_PATH + 'prod_feats.pkl')

# 2. 데이터 병합하고 훈련/테스트 데이터로 분리하기

## 2.1 데이터 하나로 병합

In [13]:
user_feats = pd.read_pickle(PREPARED_PATH + 'user_feats.pkl')
user_prod_feats = pd.read_pickle(PREPARED_PATH + 'user_prod_feats.pkl')
prod_feats = pd.read_pickle(PREPARED_PATH + 'prod_feats.pkl')

In [14]:
user_prod_feats.drop(['U_total_orders', 'product_name', 'aisle_id', 'department_id', ], axis=1, inplace=True)
prod_feats.drop(['product_name', 'aisle', 'department'], axis=1, inplace=True)

prepared_df = user_prod_feats.merge(prod_feats, on='product_id', how='left')
prepared_df = prepared_df.merge(user_feats, on='user_id', how='left') # 유저 총 주문 수 중복 확인!
reduce_memory(prepared_df)

print(prepared_df.shape)
prepared_df.head()

Memory usage of properties dataframe is : 3883.5846118927  MB

___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  1522.9743576049805  MB
This is  39.21568627450981 % of the initial size
(13307953, 49)


,user_id,product_id,UP_total_orders,UP_total_reorders,UP_mean_cart_order,UP_reorder_ratio,UP_order_percentage,UP_order_days,UP_mean_days_since_prior_order,UP_order_3,...,U_unique_reordered_products,U_avg_basket_size,U_avg_reorder_in_order,U_order_days,U_basket_size_order_3,U_basket_size_order_2,U_basket_size_order_1,U_re_in_order_3,U_re_in_order_2,U_re_in_order_1
0,1,196,10,9,1.400391,1.00,1.000000,176,19.5625,1,...,10.0,5.898438,0.78418,176.0,6,6,9,0.666504,1.0,0.666504
1,1,12427,10,9,3.300781,1.00,1.000000,176,19.5625,1,...,10.0,5.898438,0.78418,176.0,6,6,9,0.666504,1.0,0.666504
2,1,10258,9,8,3.333984,1.00,0.899902,161,20.1250,1,...,10.0,5.898438,0.78418,176.0,6,6,9,0.666504,1.0,0.666504
3,1,25133,8,7,4.000000,1.00,0.799805,140,20.0000,1,...,10.0,5.898438,0.78418,176.0,6,6,9,0.666504,1.0,0.666504
4,1,13032,3,2,6.332031,0.25,0.300049,50,25.0000,-1,...,10.0,5.898438,0.78418,176.0,6,6,9,0.666504,1.0,0.666504


### 2.1.1 최종 피쳐

In [15]:
prepared_df.columns

Index(['user_id', 'product_id', 'UP_total_orders', 'UP_total_reorders',
       'UP_mean_cart_order', 'UP_reorder_ratio', 'UP_order_percentage',
       'UP_order_days', 'UP_mean_days_since_prior_order', 'UP_order_3',
       'UP_order_2', 'UP_order_1', 'P_total_orders', 'P_unique_users',
       'P_total_reorders', 'P_reorder_rate', 'P_mean_cart_order',
       'P_sum_order_days', 'P_mean_order_days', 'P_days_since_prior_order',
       'aisle_id', 'department_id', 'is_organic', 'is_lowfat', 'P_keyword',
       'A_total_orders', 'A_total_reorders', 'A_reorder_rate',
       'A_avg_cart_order', 'A_mean_days_since_prior_order', 'D_total_orders',
       'D_total_reorders', 'D_reorder_rate', 'D_avg_cart_order',
       'D_mean_days_since_prior_order', 'U_total_orders', 'U_total_products',
       'U_unique_products', 'U_total_reorders', 'U_unique_reordered_products',
       'U_avg_basket_size', 'U_avg_reorder_in_order', 'U_order_days',
       'U_basket_size_order_3', 'U_basket_size_order_2',
     

In [16]:
prepared_df.to_pickle(PREPARED_PATH + 'prepared_df.pkl')

## 2.2 훈련/테스트로 데이터 분리

In [17]:
orders = pd.read_csv(RAW_PATH + 'orders.csv', dtype={
                                'order_id': np.uint32,              # 1 ~ 3421083
                                'user_id': np.uint32,               # 1 ~ 206209
                                'eval_set': 'category',             # 3가지
                                'order_number': np.uint8,           # 1 ~ 100
                                'order_dow': np.uint8,              # 0 ~ 6 
                                'order_hour_of_day': np.uint8,      # 0 ~ 23
                                'days_since_prior_order': np.float32})
train = pd.read_csv(RAW_PATH + 'order_products__train.csv', dtype={
                               'order_id': np.uint32,          # 1 ~ 3421083
                               'product_id': np.uint16,        # 1 ~ 49688
                               'add_to_cart_order': np.uint8,  # 1 ~ 80 
                               'reordered': np.uint8})         # 0 ~ 1
prepared_df = pd.read_pickle(PREPARED_PATH + 'prepared_df.pkl')

In [18]:
# 1. train, test 셋 유저 명단 뽑기 
train_users = orders[orders.eval_set == 'train'].user_id.unique() 
test_users = orders[orders.eval_set == 'test'].user_id.unique() 

# 2. train, test set 분리
train_df = prepared_df[prepared_df['user_id'].isin(train_users)]
test_df = prepared_df[prepared_df['user_id'].isin(test_users)]

## 2.3 훈련 데이터에 타겟 추가

In [19]:
train = train.merge(orders, on='order_id', how='left')

train_df = train_df.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id','product_id'], how='left') 
train_df['reordered'].fillna(0, inplace=True)
train_df = train_df.astype({'reordered': np.uint8})

train_df.sort_values(by=['user_id', 'UP_total_reorders'], ascending=[True, False], inplace=True, ignore_index=True)
train_df.shape, test_df.shape

((8474661, 50), (4833292, 49))

In [20]:
train_df.to_pickle(PREPARED_PATH + 'train_df.pkl')
test_df.to_pickle(PREPARED_PATH + 'test_df.pkl')

In [21]:
# train_df = pd.read_pickle(PREPARED_PATH + 'train_df.pkl')
# train_df.columns